In [308]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

In [309]:
#Baseball Reference web site
url = "https://www.baseball-reference.com/register/leader.cgi?type=bat&id=17edbc3b"
#html = urlopen(url)
#soup = BeautifulSoup(html)

data = requests.get(url).text

In [311]:
pd.set_option('display.max_columns', 500)


- need to correct function parse_player_features

In [331]:
def parse_player_features(raw_data):
    
    player_dico = {}

    counter = 0

    for player_features in raw_data[1:]:
    
        counter += 1
    
    
        player_name = player_features.split('<td')[0].split('</a')[0].split('=')[3].split('>')[1]
    
    
        player_stats = BeautifulSoup(player_features,'html.parser').findAll('td')
    
        player_features = []
    
        for element in player_stats:
        
            element_parsed = str(element).split('>')[1].replace('</td','')
        
            if element_parsed != '':
            
            
        
                player_features.append(element_parsed)
        
      
        if counter == 1:
    
            player_full_data = pd.DataFrame.from_dict({'player_name': player_name, 'age': [player_features[0]], 'batting_side': [player_features[1]], 'throwing_hand': [player_features[2]], 'height': [player_features[3]], 'weight': [player_features[4]]})
    
    
        else:
        
            player_data = pd.DataFrame.from_dict({'player_name': player_name, 'age': [player_features[0]], 'batting_side': [player_features[1]], 'throwing_hand': [player_features[2]], 'height': [player_features[3]], 'weight': [player_features[4]]})
        
            player_full_data = pd.concat([player_full_data, player_data.copy()])
            
    return player_full_data

- Number of KBO teams: need to store in list

- Years: 2018 - 2024 (can go back to 1982)

- Focus on team batting statistics

- Need to store page ID for each team between 2018 and 2024

In [10]:
team_page_ids_dictionary = {'Kia Tigers': ['684c856e', 'd3dec042', 'a16474ae', '317b4154', '6dc83e6e', '202d12bd', 'bc0e9017'],
                           'Doosan Bears': ['76fc3141', '5adff64d', '55e9e1d3', '547d336d', 'ad74c922', '46fb74ab', '8b7b3d4e'],
                           'Hanwha Eagles': ['42d98d42', 'e33f131b', '520fce90','790fbcec', '1ca97895', 'bc1a116f', 'ce8be464'],
                           'Kiwoom Heroes (prev. Nexen Heroes)': ['5659b4ad', '94cebd4b', '30053a8f','25bd07a9', '70a5ad25', '5661fb24', 'b8e758fe'],
                           'KT Wiz': ['2b821d38', 'd11780ab', '5b7d63c8','4f5c4439', '86b846ee', '3a32df68', '1004ec43'],
                           'LG Twins': ['77f134cc', '680dbde1', '48a0549a','e562a921', 'd2282f35', 'a01bea66', 'adacf419'],
                           'Lotte Giants': ['7154d748', '5a5b88b3', 'dcbb26b6','7ad5b0be', '21ee3a57', 'fac2ad8a', 'fa4816af'],
                           'NC Dinos': ['fd6c0c14', 'c958ce5c', '6fb98b94','02bfe422', '806d8e66', '5e54d3e6', 'c9f805af'],
                           'Samsung Lions': ['2962dc25', '6bebcd00', 'e8c5dfdb', '9990edb8', 'ba521d76', '5f280334', '2dbad134'],
                           'SSG Landers (prev. SK Wyverns)': ['1be13c3b', '057aea68', 'd917beac','119f7034', 'b2f0c632', '7af0ce4b', 'c693fbc3']}

In [16]:
year_list = []

for year in range(2018, 2025):
    
    year_list.append(year)   

counter = 0

for kbo_team in team_page_ids_dictionary.keys():
    
    counter += 1
    
    kbo_data = pd.DataFrame(year_list).rename(columns = {0: 'year'})
    
    kbo_data['team'] = kbo_team
    
    kbo_data['team_season_url_page_id'] = pd.Series(team_page_ids_dictionary[kbo_team])
    
    kbo_data['team_season_url'] = 'https://www.baseball-reference.com/register/team.cgi?id=' + kbo_data['team_season_url_page_id']
    
    if counter == 1:
        
        kbo_full_data = kbo_data.copy()
        
    else:
        
        kbo_full_data = pd.concat([kbo_full_data, kbo_data.copy()])

In [22]:
kbo_full_data

,year,team,team_season_url_page_id,team_season_url
0,2018,Kia Tigers,684c856e,https://www.baseball-reference.com/register/te...
1,2019,Kia Tigers,d3dec042,https://www.baseball-reference.com/register/te...
2,2020,Kia Tigers,a16474ae,https://www.baseball-reference.com/register/te...
3,2021,Kia Tigers,317b4154,https://www.baseball-reference.com/register/te...
4,2022,Kia Tigers,6dc83e6e,https://www.baseball-reference.com/register/te...
...,...,...,...,...
2,2020,SSG Landers (prev. SK Wyverns),d917beac,https://www.baseball-reference.com/register/te...
3,2021,SSG Landers (prev. SK Wyverns),119f7034,https://www.baseball-reference.com/register/te...
4,2022,SSG Landers (prev. SK Wyverns),b2f0c632,https://www.baseball-reference.com/register/te...
5,2023,SSG Landers (prev. SK Wyverns),7af0ce4b,https://www.baseball-reference.com/register/te...


- Get the first tbody content for each page then parse

In [20]:
team_seasons_urls = list(kbo_full_data['team_season_url'])

# I) Parsing Baseball Season Stats

- You accessed more than thirty pages in less than a minute

In [382]:
def parsing_season_team_batting_table(data):
    
    player_batting_stats_dico = {}

    counter = 0

    player_names = []

    for element in data:
    
    
    
        str_element = str(element)
    
    #print(str_element)
    
        if 'player.fcgi?id=' in str_element:
        
        #print(str_element)
        

        
            player_name = str_element.split('player.fcgi?id=')[2].split('>')[1].replace('</a','')
        
            player_names.append(player_name)
        
                
            if counter == 1:
            
                print(counter)
            
                full_player_batting_stats = pd.DataFrame(player_batting_stats_dico)
            
            #full_player_batting_stats['player_name'] = player_name
            
            elif counter > 1:
            
                player_batting_stats = pd.DataFrame(player_batting_stats_dico)
            
            #player_batting_stats['player_name'] = player_name
            
                full_player_batting_stats = pd.concat([full_player_batting_stats, player_batting_stats])
            
                player_batting_stats_dico = {}
        
                player_name = str_element.split('player.fcgi?id=')[2].split('>')[1].replace('</a','')
        
                player_stats = ''
            
            #full_player_batting_stats['player_name'] = player_name
            
            elif counter == 0:
            
        
                player_batting_stats_dico = {}
        
                player_name = str_element.split('player.fcgi?id=')[2].split('>')[1].replace('</a','')
        
                player_stats = ''
        
        #print(name)
        
            counter += 1
        
        else:
        
            kpi = str_element.split('data-stat="')[1].split('>')[0].replace('"','')
        
            kpi_value = [str_element.split('data-stat="')[1].split('>')[1].split('<')[0]]
        
        #print(kpi)
        
        #print(kpi_value)
        
            player_batting_stats_dico[kpi] = kpi_value
        
    
    player_batting_stats = pd.DataFrame(player_batting_stats_dico)

    full_player_batting_stats = pd.concat([full_player_batting_stats, player_batting_stats])

    full_player_batting_stats = pd.merge(full_player_batting_stats.reset_index().reset_index().drop('index', axis=1).rename(columns = {'level_0': 'index'}), pd.Series(player_names).to_frame().reset_index(), on = 'index')

    full_player_batting_stats = full_player_batting_stats.rename(columns = {0: 'player_name'})
    
    return full_player_batting_stats

In [418]:
page_counter = 0

for team_season in team_seasons_urls:
    
    filtered_data = kbo_full_data[kbo_full_data['team_season_url']==team_season]
    
    year = filtered_data['year'].values[0]
    
    team = filtered_data['team'].values[0]
    
    page_counter += 1
    
    raw_data = requests.get(team_season).text
    
    raw_data_bis = BeautifulSoup(raw_data)
    
    team_season_batting_stats = parsing_season_team_batting_table(data = raw_data_bis.findAll('tbody', limit=2)[0].findAll('td'))


    time.sleep(10)

    
    team_season_batting_stats['team'] = team
        
    team_season_batting_stats['year'] = year
    
    if page_counter == 1:
        
        
        full_team_season_batting_stats = team_season_batting_stats.copy()
        
    else:
        
        full_team_season_batting_stats = pd.concat([full_team_season_batting_stats, team_season_batting_stats.copy()])

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [2]:
#full_team_season_batting_stats

In [ ]:
#full_team_season_batting_stats.to_csv('full_kbo_team_season_batting_stats_bis.csv', index=False)

# II) Parsing Player Features (age, weight, handedness)

In [335]:
counter_page = 0

for team_season_url in kbo_full_data['team_season_url']:
    
    counter_page += 1
    
    kbo_full_data_filtered = kbo_full_data[kbo_full_data['team_season_url']==team_season_url]
    
    year = kbo_full_data_filtered['year'].values[0]
    
    team = kbo_full_data_filtered['team'].values[0]
    
    time.sleep(10)
    
    print(team)
    
    print(counter_page)
    
    raw_data = requests.get(team_season_url).text
    raw_data_bis = raw_data.split('div_standard_roster')[1].split('tbody')[1].split('data-stat="player"')
    
    player_feature_season = parse_player_features(raw_data = raw_data_bis)
    
    player_feature_season['year'] = year
    
    player_feature_season['team'] = team
    
    if counter_page == 1:
        
        player_feature_full_seasons = player_feature_season.copy()
        
    else:
        
        player_feature_full_seasons = pd.concat([player_feature_full_seasons, player_feature_season.copy()])

Kia Tigers
1
Kia Tigers
2
Kia Tigers
3
Kia Tigers
4
Kia Tigers
5
Kia Tigers
6
Kia Tigers
7
Doosan Bears
8
Doosan Bears
9
Doosan Bears
10
Doosan Bears
11
Doosan Bears
12
Doosan Bears
13
Doosan Bears
14
Hanwha Eagles
15
Hanwha Eagles
16
Hanwha Eagles
17
Hanwha Eagles
18
Hanwha Eagles
19
Hanwha Eagles
20
Hanwha Eagles
21
Kiwoom Heroes (prev. Nexen Heroes)
22
Kiwoom Heroes (prev. Nexen Heroes)
23
Kiwoom Heroes (prev. Nexen Heroes)
24
Kiwoom Heroes (prev. Nexen Heroes)
25
Kiwoom Heroes (prev. Nexen Heroes)
26
Kiwoom Heroes (prev. Nexen Heroes)
27
Kiwoom Heroes (prev. Nexen Heroes)
28
KT Wiz
29
KT Wiz
30
KT Wiz
31
KT Wiz
32
KT Wiz
33
KT Wiz
34
KT Wiz
35
LG Twins
36
LG Twins
37
LG Twins
38
LG Twins
39
LG Twins
40
LG Twins
41
LG Twins
42
Lotte Giants
43
Lotte Giants
44
Lotte Giants
45
Lotte Giants
46
Lotte Giants
47
Lotte Giants
48
Lotte Giants
49
NC Dinos
50
NC Dinos
51
NC Dinos
52
NC Dinos
53
NC Dinos
54
NC Dinos
55
NC Dinos
56
Samsung Lions
57
Samsung Lions
58
Samsung Lions
59
Samsung Lions

In [339]:
player_feature_full_seasons

,player_name,age,batting_side,throwing_hand,height,weight,year,team
0,Chi Hong An,27,R,R,"5' 10""",194,2018,Kia Tigers
0,Yong Hwan Baik,29,R,R,"5' 10""",209,2018,Kia Tigers
0,Roger Bernadina,34,L,L,"6' 2""",210,2018,Kia Tigers
0,Hyoung Woo Choi,34,L,R,"5' 10""",233,2018,Kia Tigers
0,Jung Min Choi,29,L,R,"5' 9""",158,2018,Kia Tigers
...,...,...,...,...,...,...,...,...
0,Jin Yong Seo,31,R,R,"6' 0""",194,2024,SSG Landers (prev. SK Wyverns)
0,Beom Soo Shin,26,L,R,"5' 9""",182,2024,SSG Landers (prev. SK Wyverns)
0,Heon Min Shin,21,R,R,"6' 1""",180,2024,SSG Landers (prev. SK Wyverns)
0,Keisho Shirakawa,23,R,R,"5' 10""",194,2024,SSG Landers (prev. SK Wyverns)


In [341]:
#player_feature_full_seasons.to_csv('player_features_baseball.csv', index=False)

id : 
    
- all_standard_roster

- div_standard_roster


Then parse: 

- tbody

Columns: 
    
- G -- Games Played or Pitched



- PA -- Plate Appearances


When available, we use actual plate appearances from play-by-play game accounts


Otherwise estimated using AB + BB + HBP + SF + SH,


which excludes catcher interferences.



- AB -- At Bats


At-bats are used as the denominator when determining batting average and slugging percentage


- R -- Runs Scored/Allowed


A player is awarded a run if he crosses the plate to score his team a run



- H -- Hits/Hits Allowed



A hit occurs when a batter strikes the baseball into fair territory and reaches base without doing so via an error or a fielder's choice. 


There are four types of hits in baseball: singles, doubles, triples and home runs. 


All four are counted equally when deciphering batting average. 


If a player is thrown out attempting to take an extra base (e.g., turning a single into a double), that still counts as a hit.




- 2B -- Doubles Hit/Allowed



A batter is credited with a double when he hits the ball into play and reaches second base without the help of an intervening error or attempt to put out another baserunner.





- 3B -- Triples Hit/Allowed


a triple occurs when a batter hits the ball into play and reaches third base without the help of an intervening error or attempt to put out another baserunner.





- HR -- Home Runs Hit/Allowed



A home run occurs when a batter hits a fair ball and scores on the play without being put out or without the benefit of an error.



In almost every instance of a home run, a batter hits the ball in the air over the outfield fence in fair territory. In that situation, the batter is awarded all four bases, and any runners on base score as well



- HRR (Home Run Rate = Home Run %) = number of Home Runs / number of AB (At Bats)



- RBI -- Runs Batted In


A batter is credited with an RBI in most cases where the result of his plate appearance is a run being scored.


- SB -- Stolen Bases


A stolen base occurs when a baserunner advances by taking a base to which he isn't entitled


- CS -- Caught Stealing



A caught stealing occurs when a runner attempts to steal but is tagged out before reaching second base, third base or home plate. This typically happens after a pitch, when a catcher throws the ball to the fielder at the base before the runner reaches it


- BB -- Bases on Balls/Walks


A walk (or base on balls) occurs when a pitcher throws four pitches out of the strike zone, none of which are swung at by the hitter.


- SO -- Strikeouts


A strikeout occurs when a pitcher throws any combination of three swinging or looking strikes to a hitter. (A foul ball counts as a strike, but it cannot be the third and final strike of the at-bat. A foul tip, which is caught by the catcher, is considered a third strike.)

The batter is automatically out on a strikeout, unless the catcher does not cleanly hold onto the baseball or if the baseball hits the dirt. If the catcher does not catch the third strike, the batter may attempt to run to first base -- if it is open or if there are two outs. However, even if the batter reaches first base safely, the pitcher and the batter are still credited with a strikeout in the scorebook.



- BA -- Hits/At Bats

= Batting Average

For recent years, leaders need 3.1 PA
per team game played
Bold indicates highest BA using current stats
Gold means awarded title at end of year.


- OBP -- (H + BB + HBP)/(At Bats + BB + HBP + SF)

= On-Base %

OBP refers to how frequently a batter reaches base per plate appearance

For recent years, leaders need 3.1 PA
per team game played


- SLG -- Total Bases/At Bats or (1B + 2*2B + 3*3B + 4*HR)/AB

For recent years, leaders need 3.1 PA
per team game played


= Slugging percentage



= total number of bases a player records per at-bat. Unlike on-base percentage, slugging percentage deals only with hits and does not include walks and hit-by-pitches in its equation.


- OPS -- On-Base + Slugging Percentages

OPS adds on-base percentage and slugging percentage to get one number that unites the two. It's meant to combine how well a hitter can reach base, with how well he can hit for average and for power.




For recent years, leaders need 3.1 PA
per team game played


- TB -- Total Bases
Singles + 2 x Doubles + 3 x Triples + 4 x Home Runs.


Total bases refer to the number of bases gained by a batter through his hits.


- GDP (Ground Into Double Play) -- Double Plays Grounded Into
Only includes standard 6-4-3, 4-3, etc. double plays.
First tracked in 1933.
For gamelogs only in seasons we have play-by-play, we include triple plays as well.
All official seasonal totals do not include GITP's.


A GIDP occurs when a player hits a ground ball that results in multiple outs on the bases. 


The most common double plays are ground balls where a forceout is made on the player running from first to second base, then another forceout is made on the batter running to first base.


- HBP -- Times Hit by a Pitch.



A hit-by-pitch occurs when a batter is struck by a pitched ball without swinging at it. He is awarded first base as a result. Strikes supersede hit-by-pitches, meaning if the umpire rules that the pitch was in the strike zone or that the batter swung, the HBP is nullified.

- SH -- Sacrifice Hits (Sacrifice Bunts)


A sacrifice fly occurs when a batter hits a fly-ball out to the outfield or foul territory that allows a runner to score. 


The batter is given credit for an RBI


- SF -- Sacrifice Flies
First tracked in 1954.

A sacrifice fly does not count as an at-bat and therefore does not count against a player's batting average.


- IBB -- Intentional Bases on Balls
First tracked in 1955.


An intentional walk occurs when the defending team elects to walk a batter on purpose, putting him on first base instead of letting him try to hit. Intentional walks -- which count as a walk for the hitter and a walk allowed by the pitcher -- are an important strategy in the context of a game. They can be used to put a runner on first base, setting up a potential double play.



- Hard Hit Rate = percentage of batted balls that were hit at 95 mph or more

In [383]:
#full_team_season_batting_stats.to_csv('full_kbo_team_season_batting_stats.csv', index=False)

In [392]:
all_data = pd.read_csv('full_kbo_team_season_batting_stats.csv')

In [478]:
batters_data = pd.read_csv('batter_data.csv')